# Introducción 

# Objetivos 

El objetivo general de la prueba `recommender_system_test` es evaluar el impacto de un sistema de recomendaciones mejorado en el comportamiento de los usuarios en términos de conversión a través del embudo de pago. La prueba involucra dos grupos: el grupo de control (A) y el grupo experimental con el nuevo embudo de pago (B). La prueba se lanzó el 7 de diciembre de 2020, dejó de aceptar nuevos usuarios el 21 de diciembre de 2020 y finalizó el 1 de enero de 2021. La audiencia de la prueba consiste en el 15% de los nuevos usuarios de la región de la UE, con un número previsto de 6,000 participantes. El propósito de la prueba es medir si, dentro de los 14 días posteriores a la inscripción, los usuarios muestran una mejor conversión en vistas de la página del producto (`product_page`), en la acción de agregar artículos al carrito de compras (`product_cart`) y en las compras (`purchase`). Se espera que en cada etapa del embudo (`product_page → product_cart → purchase`), haya al menos un aumento del 10%. Los datos de la prueba deben descargarse, comprobarse para asegurar que la prueba se realizó correctamente, y analizarse para evaluar los resultados obtenidos.


# Descomposición de la tarea

* ...

# Carga de librerías 

In [2]:
# Carga de librerías 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go

# Carga de datos

In [3]:
# Carga de datasets desde carpeta local
data_path = os.path.join("..", "datasets")

dataset1_path = os.path.join(data_path, "ab_project_marketing_events_us.csv")
dataset2_path = os.path.join(data_path, "final_ab_events_upd_us.csv")
dataset3_path = os.path.join(data_path, "final_ab_new_users_upd_us.csv")
dataset4_path = os.path.join(data_path, "final_ab_participants_upd_us.csv")

# el calendario de eventos de marketing para 2020
ab_project_marketing_data = pd.read_csv(dataset1_path)

# todos los usuarios que se registraron en línea desde 7 al 21 de diciembre
final_ab_new_events_data = pd.read_csv(dataset3_path)

# todos los eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021
final_ab_events_data = pd.read_csv(dataset2_path)

# tabla con los datos de los participantes de la prueba
final_ab_participants_data = pd.read_csv(dataset4_path)

In [4]:
# Carga de datos en jupyter de platforma

"""ab_project_marketing_data = pd.read_csv("/datasets/ab_project_marketing_events_us.csv")
final_ab_events_data = pd.read_csv("/datasets/final_ab_events_upd_us.csv")
final_ab_new_events_data = pd.read_csv("/datasets/final_ab_new_users_upd_us.csv")
final_ab_participants_data = pd.read_csv("/datasets/final_ab_participants_upd_us.csv")"""

'ab_project_marketing_data = pd.read_csv("/datasets/ab_project_marketing_events_us.csv")\nfinal_ab_events_data = pd.read_csv("/datasets/final_ab_events_upd_us.csv")\nfinal_ab_new_events_data = pd.read_csv("/datasets/final_ab_new_users_upd_us.csv")\nfinal_ab_participants_data = pd.read_csv("/datasets/final_ab_participants_upd_us.csv")'

## Exploración inicial de datos

### Revisión de dataset ab_project_marketing_data

In [5]:
# Revisión de datos ab_project_data
print('- Información sobre dataset -')
print(ab_project_marketing_data.head(5))
print()
print(ab_project_marketing_data.info())
print()
print('- Datos ausentes -')
print(ab_project_marketing_data.isna().sum())
print()
print('- Valores duplicados -')
print(ab_project_marketing_data.duplicated().sum())

- Información sobre dataset -
                           name                   regions    start_dt  \
0      Christmas&New Year Promo             EU, N.America  2020-12-25   
1  St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14   
2        St. Patric's Day Promo             EU, N.America  2020-03-17   
3                  Easter Promo  EU, CIS, APAC, N.America  2020-04-12   
4             4th of July Promo                 N.America  2020-07-04   

    finish_dt  
0  2021-01-03  
1  2020-02-16  
2  2020-03-19  
3  2020-04-19  
4  2020-07-11  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes
None

- Datos ausentes -
name         0
regions      

- `name`: el nombre del evento de marketing
- `regions`: regiones donde se llevará a cabo la campaña publicitaria
- `start_dt`: fecha de inicio de la campaña
- `finish_dt`: fecha de finalización de la campaña

In [6]:
# Cambio de formato de columna start_dt
ab_project_marketing_data['start_dt'] = pd.to_datetime(ab_project_marketing_data['start_dt'])
ab_project_marketing_data['finish_dt'] = pd.to_datetime(ab_project_marketing_data['finish_dt'])

### Revisión de dataset final_ab_events_data

In [7]:
# Revisión de dataset final_ab_events_data
print('- Información sobre dataset -')
print(final_ab_events_data.head())
print()
print(final_ab_events_data.info())
print()
print('- Datos ausentes -')
print(final_ab_events_data.isna().sum())
print()
print('- Valores duplicados -')
print(final_ab_events_data.duplicated().sum())

- Información sobre dataset -
            user_id             event_dt event_name  details
0  E1BDDCE0DAFA2679  2020-12-07 20:22:03   purchase    99.99
1  7B6452F081F49504  2020-12-07 09:22:53   purchase     9.99
2  9CD9F34546DF254C  2020-12-07 12:59:29   purchase     4.99
3  96F27A054B191457  2020-12-07 04:02:40   purchase     4.99
4  1FD7660FDF94CA1F  2020-12-07 10:15:09   purchase     4.99



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB
None

- Datos ausentes -
user_id            0
event_dt           0
event_name         0
details       363447
dtype: int64

- Valores duplicados -
0


- `user_id`
- `event_dt`: fecha y hora del evento
- `event_name`: nombre del tipo de evento
- `details`: datos adicionales sobre el evento (por ejemplo, el pedido total en USD para los eventos `purchase`)

In [8]:
# Cambio de formato de columna event_date
final_ab_events_data['event_dt'] = pd.to_datetime(final_ab_events_data['event_dt'])

In [9]:
# Revisión de datos en la columna 'details'
final_ab_events_data['details'].sample(10)

350313     NaN
191338     NaN
419661     NaN
161323     NaN
144141     NaN
2750      4.99
201640     NaN
258155     NaN
219209     NaN
72240      NaN
Name: details, dtype: float64

In [10]:
# Revisión de datos que contienen 'NaN' en la columna 'details'
no_purchase = final_ab_events_data[final_ab_events_data['details'].isna()]
no_purchase.sample(10)

# Se aplica reemplazo de valores NaN por 0 
final_ab_events_data.fillna({'details':0}, inplace = True)
print(final_ab_events_data.sample(10))

                 user_id            event_dt    event_name  details
265054  472847FDBF859388 2020-12-11 19:48:32         login     0.00
115426  0F1E967B3A74EF69 2020-12-26 22:12:04  product_cart     0.00
398223  6E85B86178B06FFD 2020-12-24 15:38:19         login     0.00
131211  1B5FB41133E3E5EE 2020-12-10 18:45:45  product_page     0.00
274341  A7B1E841B8A61D7C 2020-12-12 01:41:52         login     0.00
33718   910B74369244BAB3 2020-12-19 20:24:56      purchase     4.99
298355  4DE69C374C603590 2020-12-15 14:01:22         login     0.00
262809  BB3B25FD41E6A916 2020-12-10 06:05:00         login     0.00
308559  CE4FBBCF30FBE571 2020-12-16 11:07:47         login     0.00
260859  CCB80CA83873D13D 2020-12-10 21:41:36         login     0.00


In [11]:
# Verificación de los valores de eventos
final_ab_events_data['event_name'].value_counts()

event_name
login           182465
product_page    120862
purchase         60314
product_cart     60120
Name: count, dtype: int64

Columna 'details' contiene información numérica y NaN, la documentación del dataset indica que tiene la cantidad en USD para cuando el evento es 'purchase'. Se infiere que los datos NaN son 0, ya que no representan una transacción concretada. 
Se observa que hay más valores de `purchase` que de `product_cart`, lo que puede sugerir que se evita esta última etapa y se puede comprar con algo parecido a un botón 'compra rápida'

### Revisión de dataset final_ab_new_events_data

In [12]:
# Revisión de datos en dataset final_ab_new_events_data
print('- Información sobre dataset -')
print(final_ab_new_events_data.sample(10))
print()
print(final_ab_new_events_data.info())
print()
print('- Datos ausentes -')
print(final_ab_new_events_data.isna().sum())
print()
print('- Valores duplicados -')
print(final_ab_new_events_data.duplicated().sum())

- Información sobre dataset -
                user_id  first_date     region   device
11474  E084BB639B87D0DF  2020-12-21         EU   iPhone
11166  15B4E2FD960233DC  2020-12-21         EU      Mac
29159  6B88673EF5EF2B81  2020-12-16         EU      Mac
30114  337CF51CF67A9304  2020-12-23         EU       PC
30578  E1C561657C272A8A  2020-12-23         EU   iPhone
5447   AC96D1F8841D59D2  2020-12-14         EU       PC
37033  00D627B7002B66CB  2020-12-17         EU  Android
4052   4FDA5266CAC1EE4F  2020-12-07         EU   iPhone
51861  C9A18EE340407344  2020-12-13  N.America  Android
46618  5800C72FA6246FD6  2020-12-12         EU  Android

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtype

- `user_id`
- `first_date`: fecha de inscripción
- `region`
- `device`: dispositivo utilizado para la inscripción

In [13]:
# Se cambia el formato de la columna 'first_date' 
final_ab_new_events_data['first_date'] = pd.to_datetime(final_ab_new_events_data['first_date'])


No contiene datos nulos y el único cambio realizado fue el formato de fecha en la columna 'first_date'

### Revisión del dataset final_ab_participants_data

In [14]:
# Revisión del dataset 

print('- Información sobre dataset -')
print(final_ab_participants_data.head())
print()
print(final_ab_participants_data.info())
print()
print('- Datos ausentes -')
print(final_ab_participants_data.isna().sum())
print()
print('- Valores duplicados -')
print(final_ab_participants_data.duplicated().sum())

- Información sobre dataset -
            user_id group                  ab_test
0  D1ABA3E2887B6A73     A  recommender_system_test
1  A7A3664BD6242119     A  recommender_system_test
2  DABC14FDDFADD29E     A  recommender_system_test
3  04988C5DF189632E     A  recommender_system_test
4  4FF2998A348C484F     A  recommender_system_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB
None

- Datos ausentes -
user_id    0
group      0
ab_test    0
dtype: int64

- Valores duplicados -
0


- `user_id`
- `ab_test`: nombre de la prueba
- `group`: el grupo de prueba al que pertenecía el usuario

# Segmentación de datos para realizar test A/B

Se requieren los datos que cumplan con las siguientes condiciones:
* Usuarios que se hayan inscrito entre 7 (inicio de la prueba) y 18 de diciembre (14 días antes de la fecha de finalización 01-01-2021). 
* Que pertenezcan a la región de UE
* Valor de la columna 'ab_test' = "recommender_system_test"
* Nombre del evento por usuario.
* El tiempo desde la inscripción del usuario hasta fecha de eventos este dentro de 14 días. 

In [15]:
# Generación de dataset de trabajo con las columnas requeridas

wdataset = pd.merge(final_ab_participants_data, final_ab_new_events_data, on='user_id', how = 'inner')
wdataset = pd.merge(wdataset, final_ab_events_data, on = 'user_id', how = 'inner')

# Generación de columna con día del evento 
wdataset['event_date'] = wdataset['event_dt'].dt.date
wdataset

,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details,event_date
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,purchase,99.99,2020-12-07
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:29,product_cart,0.00,2020-12-07
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,product_page,0.00,2020-12-07
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,login,0.00,2020-12-07
4,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone,2020-12-20 15:46:06,product_page,0.00,2020-12-20
...,...,...,...,...,...,...,...,...,...,...
102833,63FBE257B05F2245,A,interface_eu_test,2020-12-20,EU,Android,2020-12-23 13:03:17,login,0.00,2020-12-23
102834,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,2020-12-14 19:12:49,login,0.00,2020-12-14
102835,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,2020-12-15 01:50:22,login,0.00,2020-12-15
102836,79F9ABFB029CF724,B,interface_eu_test,2020-12-14,EU,PC,2020-12-16 19:27:36,login,0.00,2020-12-16


Data set de trabajo contiene los grupos, ab_test, fecha de inscripción, región, dispositivo, fecha del evento, nombre del evento, y detalles 

Data set contiene 102,833 registros.

In [16]:
# Número único de usuarios

wdataset['user_id'].nunique()

13638

Se tiene 13,638 usuarios únicos. 

In [17]:
# Filtrado de datos tomando en cuenta los requerimientos del estudio 

wdataset_filtered = wdataset[(wdataset['ab_test'] == 'recommender_system_test') &
                             (wdataset['first_date'] <= '2020-12-21') &
                             (wdataset['region'] == 'EU')]

wdataset_filtered

,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details,event_date
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,purchase,99.99,2020-12-07
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:29,product_cart,0.00,2020-12-07
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,product_page,0.00,2020-12-07
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,login,0.00,2020-12-07
4,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone,2020-12-20 15:46:06,product_page,0.00,2020-12-20
...,...,...,...,...,...,...,...,...,...,...
22669,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-15 20:10:06,product_page,0.00,2020-12-15
22670,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-20 20:01:34,product_page,0.00,2020-12-20
22671,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-14 05:48:50,login,0.00,2020-12-14
22672,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-15 20:10:03,login,0.00,2020-12-15


In [18]:
# Se crea una columna para calcular el tiempo de conversión entre 'first_date' y 'event_date'
wdataset_filtered['event_date'] = pd.to_datetime(wdataset_filtered['event_date'])
wdataset_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22674 entries, 0 to 22673
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     22674 non-null  object        
 1   group       22674 non-null  object        
 2   ab_test     22674 non-null  object        
 3   first_date  22674 non-null  datetime64[ns]
 4   region      22674 non-null  object        
 5   device      22674 non-null  object        
 6   event_dt    22674 non-null  datetime64[ns]
 7   event_name  22674 non-null  object        
 8   details     22674 non-null  float64       
 9   event_date  22674 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), object(6)
memory usage: 1.9+ MB


C:\Users\ibtjl\AppData\Local\Temp\ipykernel_11596\2012580890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdataset_filtered['event_date'] = pd.to_datetime(wdataset_filtered['event_date'])


In [19]:
wdataset_filtered.loc[:,'days_to_conversion'] = (wdataset_filtered['event_date'] - wdataset_filtered['first_date'])
wdataset_filtered.loc[:,'days_to_conversion'] = wdataset_filtered['days_to_conversion'].dt.days.astype('int')
wdataset_filtered

C:\Users\ibtjl\AppData\Local\Temp\ipykernel_11596\633491194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdataset_filtered.loc[:,'days_to_conversion'] = (wdataset_filtered['event_date'] - wdataset_filtered['first_date'])
C:\Users\ibtjl\AppData\Local\Temp\ipykernel_11596\633491194.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 1 6]' has dtype incompatible with timedelta64[ns], please explicitly cast to a compatible dtype first.
  wdataset_filtered.loc[:,'days_to_conversion'] = wdataset_filtered['days_to_conversion'].dt.days.astype('int')


,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details,event_date,days_to_conversion
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,purchase,99.99,2020-12-07,0
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:29,product_cart,0.00,2020-12-07,0
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,product_page,0.00,2020-12-07,0
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,login,0.00,2020-12-07,0
4,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone,2020-12-20 15:46:06,product_page,0.00,2020-12-20,0
...,...,...,...,...,...,...,...,...,...,...,...
22669,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-15 20:10:06,product_page,0.00,2020-12-15,1
22670,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-20 20:01:34,product_page,0.00,2020-12-20,6
22671,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-14 05:48:50,login,0.00,2020-12-14,0
22672,80712ED4EA1B52A5,A,recommender_system_test,2020-12-14,EU,Android,2020-12-15 20:10:03,login,0.00,2020-12-15,1


In [20]:
wdataset_filtered['days_to_conversion'].value_counts()

days_to_conversion
0     7719
1     3559
2     2451
3     1712
4     1217
5     1068
6      926
7      853
8      686
9      524
10     414
12     336
11     250
13     237
14     205
15     156
16      75
17      67
20      62
19      54
18      39
21      31
22      29
23       4
Name: count, dtype: int64

In [23]:
# Filtrado de datos usando la condición de 'days_to_conversion' <= 14
wdataset_filtered = wdataset_filtered.query("days_to_conversion <= 14")
wdataset_filtered['days_to_conversion'].value_counts()

days_to_conversion
0     7719
1     3559
2     2451
3     1712
4     1217
5     1068
6      926
7      853
8      686
9      524
10     414
12     336
11     250
13     237
14     205
Name: count, dtype: int64

In [24]:
# Revisión de distribución de usuarios por grupo 
wdataset_filtered_groups = wdataset_filtered.groupby('group').agg({'user_id':'nunique'}).reset_index()
wdataset_filtered_groups

,group,user_id
0,A,2604
1,B,877


In [25]:
final_ab_participants_data

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test
...,...,...,...
14520,1D302F8688B91781,B,interface_eu_test
14521,3DE51B726983B657,A,interface_eu_test
14522,F501F79D332BE86C,A,interface_eu_test
14523,63FBE257B05F2245,A,interface_eu_test


In [28]:
final_ab_filter_by_experiment = final_ab_participants_data[final_ab_participants_data['ab_test'] == 'recommender_system_test']
final_ab_filter_by_experiment['group'].value_counts()

group
A    2747
B     928
Name: count, dtype: int64

Sigue existiendo una diferencia considerable en la cantidad de usuarios en cada experimento